In [1]:
#聲音處理套件
import librosa
import librosa.display
#資料處理套件
import pandas as pd
import numpy as np
import torch

In [2]:
#音訊資訊CSV檔位置
CSV_FILE_PATH = "D:/dataset/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/dataset/ESC-50-master/ESC-50-master/audio/"

In [3]:
#讀取CSV檔
df = pd.read_csv(CSV_FILE_PATH)
df.head()

#刪去無用欄位
df = df.drop(['esc10','src_file','take'], axis=1)

#取出類別，去除重複並排序
classes = df['category'].unique()
class_dict = {i:x for x,i in enumerate(classes)}
print(class_dict)

{'dog': 0, 'chirping_birds': 1, 'vacuum_cleaner': 2, 'thunderstorm': 3, 'door_wood_knock': 4, 'can_opening': 5, 'crow': 6, 'clapping': 7, 'fireworks': 8, 'chainsaw': 9, 'airplane': 10, 'mouse_click': 11, 'pouring_water': 12, 'train': 13, 'sheep': 14, 'water_drops': 15, 'church_bells': 16, 'clock_alarm': 17, 'keyboard_typing': 18, 'wind': 19, 'footsteps': 20, 'frog': 21, 'cow': 22, 'brushing_teeth': 23, 'car_horn': 24, 'crackling_fire': 25, 'helicopter': 26, 'drinking_sipping': 27, 'rain': 28, 'insects': 29, 'laughing': 30, 'hen': 31, 'engine': 32, 'breathing': 33, 'crying_baby': 34, 'hand_saw': 35, 'coughing': 36, 'glass_breaking': 37, 'snoring': 38, 'toilet_flush': 39, 'pig': 40, 'washing_machine': 41, 'clock_tick': 42, 'sneezing': 43, 'rooster': 44, 'sea_waves': 45, 'siren': 46, 'cat': 47, 'door_wood_creaks': 48, 'crickets': 49}


In [4]:
class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

In [9]:
#ESC資料集聲音處理
def trainPreprocess(fold, soundLength):
    #聲音長度
    soundLength = soundLength
    #一個聲音切成幾段
    soundCount = 6 - soundLength
    
    #要回傳的資料
    #資料 ESC50標籤
    x_train, y_train = [], []
    x_val, y_val= [], []
    
    #根據fold來切分train/val
    train_df = df[df.fold != fold]
    val_df = df[df.fold == fold]
    
    #轉換train資料/添加標籤
    for data in train_df.itertuples():
        sig , sr = librosa.load(DATA_PATH+data[1], sr=conf.sr)
        for i in range(soundCount):
            mel_spec = sig[i*sr : int((i+soundLength)*sr)]
            #資料
            x_train.append(mel_spec)
            #標籤
            y_train.append(data[3])
            
    #轉換val資料/添加標籤
    for data in val_df.itertuples():
        sig , sr = librosa.load(DATA_PATH+data[1], sr=conf.sr)
        for i in range(soundCount):
            mel_spec = sig[i*sr : int((i+soundLength)*sr)]
            #資料
            x_val.append(mel_spec)
            #標籤
            y_val.append(data[3])
          
    #通通轉成 torch 的 tensor 形式
    x_train = torch.Tensor(x_train)
    y_train = torch.Tensor(y_train).type(torch.LongTensor)
    x_val = torch.Tensor(x_val) 
    y_val = torch.Tensor(y_val).type(torch.LongTensor)
    
    #轉維度 BCW
    x_train = torch.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_val = torch.reshape(x_val, (x_val.shape[0], 1, x_val.shape[1]))
    
    #存檔位置
    ESC_DATA_PATH = "D:/paper/code/Preprocessing/ESC50_1D/Fold"
    x_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_train.pt"
    y_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train.pt" 
    x_val_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_val.pt" 
    y_val_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_val.pt" 
    
    torch.save(x_train, x_train_path)
    torch.save(y_train, y_train_path)
    torch.save(x_val, x_val_path)
    torch.save(y_val, y_val_path)

In [10]:
for i in range(1,6):
    for s in range(1,6):
        trainPreprocess(i,s)

NameError: name 'librosa' is not defined